In [1]:
import pandas as pd
import re
import sys

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
indel_file ="SOMOZACOL2.combined.hf.vcf"

In [4]:
def calculate_ALT_AD(row):
    split_AD = row.AD.split(",")[1:]
    split_AD = [int(x) for x in split_AD]
    if row.len_AD > 2:
        max_AD = max(split_AD)
        #max_index = split_AD.index(max(split_AD))
        return max_AD
    else:
        #ALT_AD = row.AD.split(",")[1]
        return split_AD[0]

In [5]:
def calculate_AD_list(row):
    split_AD = row.AD.split(",")[1:]
    split_AD = [int(x) for x in split_AD]
    max_AD = max(split_AD)
    row.AD_list = split_AD
    #max_index = split_AD.index(max(split_AD))
    return split_AD

In [6]:
def handle_polymorphism(vcf_df):
    for index, _ in vcf_df[vcf_df.len_AD > 2].iterrows():
        split_AD = vcf_df.loc[index, 'AD'].split(",")[1:]
        split_AD = [int(x) for x in split_AD]
        if min(split_AD) == 0:
            max_index = split_AD.index(max(split_AD)) #Obtain index from highest value in list of positions
            split_ALT = vcf_df.loc[index, 'ALT'].split(",") #split bases into list
            vcf_df.loc[index, 'ALT'] = split_ALT[max_index] #Retrieve the base using ps index
            vcf_df.loc[index, 'len_AD'] = 2 #reset number of alternatives as normal
        elif max(split_AD)/min(split_AD) > 1.5: # the same process to avoid dividing by 0
            max_index = split_AD.index(max(split_AD))
            split_ALT = vcf_df.loc[index, 'ALT'].split(",")
            vcf_df.loc[index, 'ALT'] = split_ALT[max_index]
            vcf_df.loc[index, 'len_AD'] = 2

In [52]:
def define_var_type(row):
    len_ref = len(row.REF)
    len_alt = len(row.ALT)
    
    if len_ref == len_alt == 1:
        return "SNP"
    else:
        return "INDEL"
    

In [55]:
def import_VCF42_to_pandas(vcf_file, sep='\t'):
    """
    Script to read vcf 4.2
    - now handle correct allele frequency calculated by summing REF reads + ALT reads instead from DP parameter
    - now retrieve the largest read number for ALT allele frequency in case is a heterozygous SNP (depends on calculate_ALT_AD())
    - now uses dataframe.iterrows() instead dataframe.index
    - remove snps with two alternate alleles, keeping the most abundant if this is more at least 3 times more frequent
    """

    header_lines = 0
    with open(vcf_file) as f:
        first_line = f.readline().strip()
        next_line = f.readline().strip()
        while next_line.startswith("##"):
            header_lines = header_lines + 1
            #print(next_line)
            next_line = f.readline()
    
    if first_line.endswith('VCFv4.2'):
        
        #Use first line as header
        dataframe = pd.read_csv(vcf_file, sep=sep, skiprows=[header_lines], header=header_lines)
        sample = dataframe.columns[-1]
        dataframe.rename(columns={sample:'sample'}, inplace=True)
        
        for index, data_row in dataframe.iterrows():
            info_fields = re.findall(r';*([a-zA-Z]{1,20})=', data_row.INFO)
            info_values = re.findall(r'-?\d+\.?\d*e?[+-]?\d{0,2}', data_row.INFO)
            
            format_fields = data_row['FORMAT'].split(":")
            format_values = data_row['sample'].split(":")
                                    
            for ifield, ivalue in zip(info_fields,info_values):
                dataframe.loc[index,ifield] = ivalue
                
            for ffield, fvalue in zip(format_fields,format_values):
                dataframe.loc[index,ffield] = fvalue
            
        dataframe.rename(columns={'AF':'af'}, inplace=True)
        
        dataframe['len_AD'] = dataframe['AD'].str.split(",").str.len()
        dataframe['REF_AD'] = dataframe['AD'].str.split(",").str[0]

        dataframe['ALT_AD'] = dataframe.apply(calculate_ALT_AD, axis=1)
        dataframe[['gt0','gt1']] = dataframe['GT'].str.split(r'[/|\|]', expand=True)
        
        # this step remove false snps from cohort calling and reset index
        #dataframe = dataframe[dataframe.ALT_AD > 0].reset_index(drop=True)

        handle_polymorphism(dataframe) #Leave the most common variation
        dataframe['TYPE'] = dataframe.apply(define_var_type, axis=1)
        

        to_float = ['QUAL', 'AC', 'af', 'AN', 'BaseQRankSum', 'DP', 'ExcessHet', 'FS',
       'MLEAC', 'MLEAF', 'MQ', 'MQRankSum', 'QD', 'ReadPosRankSum', 'SOR','GQ','ALT_AD', 'REF_AD']
        
        to_int = ['POS', 'len_AD', 'gt0', 'gt1']
        
        to_str = ['#CHROM','REF','ALT', 'FILTER']
        
        for column in dataframe.columns:
            if column in to_float:
                dataframe[column] = dataframe[column].astype(float)
                
        for column in dataframe.columns:
            if column in to_int:
                dataframe[column] = dataframe[column].astype(int)
                
        for column in dataframe.columns:
            if column in to_str:
                dataframe[column] = dataframe[column].astype(str)
                
        dataframe['dp'] = (dataframe['REF_AD'] + dataframe['ALT_AD'])
        dataframe['aF'] = dataframe['REF_AD']/dataframe['dp']
        dataframe['AF'] = dataframe['ALT_AD']/dataframe['dp']
        
        dataframe = dataframe.sort_values(by=['POS']).reset_index(drop=True)
        
    else:
        print("This vcf file is not v4.2")
        sys.exit(1)
           
    return dataframe

In [56]:
def add_window_distance(vcf_df, window_size=10):
    """
    Add a column indicating the maximum number of SNPs in a windows of 10 or supplied distance
    """
    list_pos = vcf_df.POS.to_list() #all positions
    set_pos = set(list_pos) #to set for later comparing
    max_pos = max(vcf_df.POS.to_list()) #max to iter over positions (independent from reference)

    all_list = list(range(1, max_pos + 1)) #create a list to slide one by one
    
    df_header = "window_" + str(window_size)

    vcf_df[df_header] = 1 #Create all 1 by default

    #Slide over windows
    for i in range(0,max_pos,1):
        window_pos = all_list[i:i+window_size] #This splits the list in windows of determined length
        set_window_pos = set(window_pos)
        #How many known positions are in every window for later clasification
        num_conglomerate = set_pos & set_window_pos
        
        if len(num_conglomerate) > 1:
            for i in num_conglomerate:
                index = vcf_df.index[vcf_df["POS"] == i][0] #Retrieve index with the known position
                if vcf_df.loc[index,df_header] < len(num_conglomerate):
                    vcf_df.loc[index,df_header] = len(num_conglomerate)

In [123]:
def add_snp_distance(vcf_df, max_length=False):
    """
    Calculate distance to the closest left and rigth SNP using a vcf imported as datafame
    Total reference length is inferred from vcf by default adding 100bp to the largest position
    in order to avoid reference parse, but it can me supplied by user
    """
    if max_length == False:
        max_length = max(vcf_df.POS.values.tolist()) + 100
        
    for index, _ in vcf_df[vcf_df.TYPE == "SNP"].iterrows():
        if index == 0:
            vcf_df.loc[index,'snp_left_distance'] = vcf_df.loc[index,'POS'] - 0
        elif index > 0:
            vcf_df.loc[index,'snp_left_distance'] = vcf_df.loc[index,'POS'] - vcf_df.loc[index - 1,'POS']
        if index == (len(vcf_df.index.values) - 1):
            vcf_df.loc[index,'snp_right_distance'] = max_length - vcf_df.loc[index,'POS']
        elif index < (len(vcf_df.index.values) - 1):
            vcf_df.loc[index,'snp_right_distance'] = vcf_df.loc[index + 1,'POS'] - vcf_df.loc[index,'POS']
            
    return vcf_df

In [140]:
def add_indel_distance(vcf_df, max_length=False):
    """
    Calculate distance to the closest left and rigth INDEL using a vcf imported as datafame
    Total reference length is inferred from vcf by default adding 100bp to the largest position
    in order to avoid reference parse, but it can me supplied by user
    """
    if max_length == False:
        max_length = max(vcf_df.POS.values.tolist()) + 100
        
    for index, data_row in vcf_df[vcf_df.TYPE == "SNP"].iterrows():
        if index > 0 and index < max(vcf_df.index) and (vcf_df.loc[index - 1,'TYPE'] == 'INDEL'):
            if index == 0:
                vcf_df.loc[index,'indel_left_distance'] = vcf_df.loc[index,'POS'] - 0
            elif index > 0:
                vcf_df.loc[index,'indel_left_distance'] = vcf_df.loc[index,'POS'] - vcf_df.loc[index - 1,'POS']
        if index > 0 and index < max(vcf_df.index) and (vcf_df.loc[index + 1,'TYPE'] == 'INDEL'):
            if (index == (len(vcf_df.index.values) - 1)):
                vcf_df.loc[index,'indel_right_distance'] = max_length - vcf_df.loc[index,'POS']
            elif (index < (len(vcf_df.index.values) - 1)):
                vcf_df.loc[index,'indel_right_distance'] = vcf_df.loc[index + 1,'POS'] - vcf_df.loc[index,'POS']

    return vcf_df

In [141]:
max(indel.index)

1554

# INDELS

In [142]:
indel = import_VCF42_to_pandas(indel_file)

In [143]:
indel.columns

Index(['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT',
       'sample', 'AC', 'af', 'AN', 'DP', 'ExcessHet', 'FS', 'MQ', 'QD', 'SOR',
       'GT', 'AD', 'GQ', 'PL', 'BaseQRankSum', 'MQRankSum', 'ReadPosRankSum',
       'PGT', 'PID', 'PS', 'len_AD', 'REF_AD', 'ALT_AD', 'gt0', 'gt1', 'TYPE',
       'dp', 'aF', 'AF'],
      dtype='object')

In [144]:
add_snp_distance(indel)
add_indel_distance(indel)

#Add info of clustered positions in sliding window
add_window_distance(indel, window_size=10)
add_window_distance(indel, window_size=20)
add_window_distance(indel, window_size=30)

In [145]:
indel[(indel.indel_left_distance < 20) | (indel.indel_right_distance < 20)].head(20)

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,TYPE,dp,aF,AF,snp_left_distance,snp_right_distance,indel_right_distance,indel_left_distance,window_10,window_20,window_30
66,MTB_anc,208322,.,G,T,13629.09,PASS,AC=2;AF=1.00;AN=2;DP=182;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,182:182:99:1|1:208316_A_AT:8152,548,0:20...",2.0,1.0,2.0,182.0,3.0103,0.000,60.00,30.55,1.840,1|1,"0,182",99.0,"8152,548,0",NaN,NaN,NaN,1|1,208316_A_AT,208316,2,0.0,182.0,1,1,SNP,182.0,0.000000,1.000000,2.0,4031.0,NaN,2.0,3,3,3
97,MTB_anc,335803,.,G,C,875.05,PASS,AC=2;AF=1.00;AN=2;DP=8;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,8:8:24:1|1:335803_G_C:285,24,0:335803",2.0,1.0,2.0,8.0,3.0103,0.000,51.09,28.86,1.426,1|1,"0,8",24.0,"285,24,0",NaN,NaN,NaN,1|1,335803_G_C,335803,2,0.0,8.0,1,1,SNP,8.0,0.000000,1.000000,160.0,7.0,7.0,NaN,2,2,2
108,MTB_anc,336325,.,T,G,331.46,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=4.27;DP=88;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:76,12:88:99:0|1:336322_CAA_C:272,0,3185:33...",1.0,0.5,2.0,88.0,4.7712,18.432,59.17,2.65,2.567,0|1,"76,12",99.0,"272,0,3185",4.270,-4.056,0.686,0|1,336322_CAA_C,336322,2,76.0,12.0,0,1,SNP,88.0,0.863636,0.136364,3.0,2.0,2.0,3.0,3,4,6
110,MTB_anc,336332,.,G,C,429.46,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=3.45;DP=85;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:72,13:85:99:0|1:336322_CAA_C:320,0,3067:33...",1.0,0.5,2.0,85.0,4.7712,23.115,59.08,3.52,2.939,0|1,"72,13",99.0,"320,0,3067",3.450,-3.206,-0.012,0|1,336322_CAA_C,336322,2,72.0,13.0,0,1,SNP,85.0,0.847059,0.152941,5.0,12.0,NaN,5.0,3,4,6
113,MTB_anc,336353,.,C,T,613.46,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=1.03;DP=79;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:64,15:79:99:0|1:336344_A_G:431,0,2726:336344",1.0,0.5,2.0,79.0,4.7712,23.622,58.89,5.11,2.727,0|1,"64,15",99.0,"431,0,2726",1.030,-3.875,0.625,0|1,336344_A_G,336344,2,64.0,15.0,0,1,SNP,79.0,0.810127,0.189873,8.0,3.0,3.0,NaN,3,5,7
116,MTB_anc,336369,.,C,T,1377.46,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=0.409;DP=81;Ex...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:67,14:81:99:0|1:336344_A_G:935,0,2760:336344",1.0,0.5,2.0,81.0,4.7712,4.501,58.04,11.29,1.122,0|1,"67,14",99.0,"935,0,2760",0.409,-4.405,-0.804,0|1,336344_A_G,336344,2,67.0,14.0,0,1,SNP,81.0,0.827160,0.172840,9.0,1.0,NaN,9.0,2,5,7
121,MTB_anc,336405,.,A,G,2101.46,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=3.88;DP=105;Ex...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:65,40:105:99:0|1:336400_C_G:1461,0,2548:33...",1.0,0.5,2.0,105.0,4.7712,5.002,56.36,13.30,1.094,0|1,"65,40",99.0,"1461,0,2548",3.880,-4.805,0.085,0|1,336400_C_G,336400,2,65.0,40.0,0,1,SNP,105.0,0.619048,0.380952,2.0,17.0,17.0,NaN,3,3,5
123,MTB_anc,336428,.,A,G,1420.46,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=5.27;DP=90;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:62,28:90:99:0|1:336422_TTG_T:989,0,2519:33...",1.0,0.5,2.0,90.0,4.7712,8.584,58.99,10.44,0.941,0|1,"62,28",99.0,"989,0,2519",5.270,-2.793,0.392,0|1,336422_TTG_T,336422,2,62.0,28.0,0,1,SNP,90.0,0.688889,0.311111,6.0,3.0,NaN,6.0,3,5,8
131,MTB_anc,336459,.,G,A,1069.29,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-8.880e-01;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:59,28:87:99:0|1:336422_TTG_T:1076,0,2471:3...",1.0,0.5,2.0,87.0,3.9794,7.617,59.14,12.29,1.058,0|1,"59,28",99.0,"1076,0,2471",-0.888,-3.311,0.454,0|1,336422_TTG_T,336422,2,59.0,28.0,0,1,SNP,87.0,0.678161,0.321839,7.0,9.0,9.0,NaN,3,6,8
134,MTB_anc,336473,.,T,G,934.29,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=3.03;DP=90;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:64,26:90:99:0|1:336422_TTG_T:941,0,2606:33...",1.0,0.5,2.0,90.0,3.9794,3.135,59.17,10.38,0.941,0|1,"64,26",99.0,"941,0,2606",3.030,-3.577,0.837,0|1,336422_TTG_T,336422,2,64.0,26.0,0,1,SNP,90.0,0.711111,0.288889,1.0,3.0,3.0,1.0,4,5,8


In [149]:
indel[indel.TYPE == 'INDEL']

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,TYPE,dp,aF,AF,snp_left_distance,snp_right_distance,indel_right_distance,indel_left_distance,window_10,window_20,window_30
25,MTB_anc,71584,.,C,CCGAGCGCTGTTCTGGCGCTAATCTGACGCTAGAATAG,6267.13,PASS,AC=2;AF=1.00;AN=2;DP=53;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,53:53:99:3299,212,0",2.0,1.0,2.0,53.0,3.0103,0.000,60.0000,30.02,3.352,1/1,"0,53",99.0,"3299,212,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,53.0,1,1,INDEL,53.0,0.000000,1.000000,NaN,NaN,NaN,NaN,1,1,1
27,MTB_anc,79504,.,TCGGTGGACC,T,3698.09,PASS,"AC=0,2;AF=0.00,1.00;AN=2;DP=59;ExcessHet=3.010...",GT:AD:DP:GQ:PL,"2/2:0,0,59:59:99:2621,2621,2621,181,181,0",0.0,2.0,0.0,59.0,2.0000,59.000,3.0103,0.00,60.000,2/2,"0,0,59",99.0,"2621,2621,2621,181,181,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,59.0,2,2,INDEL,59.0,0.000000,1.000000,NaN,NaN,NaN,NaN,1,1,1
44,MTB_anc,125830,.,G,GA,5951.13,PASS,AC=2;AF=1.00;AN=2;DP=105;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,105:105:99:3399,316,0",2.0,1.0,2.0,105.0,3.0103,0.000,60.0000,32.17,1.142,1/1,"0,105",99.0,"3399,316,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,105.0,1,1,INDEL,105.0,0.000000,1.000000,NaN,NaN,NaN,NaN,1,1,1
46,MTB_anc,131174,.,T,TG,6359.13,PASS,AC=2;AF=1.00;AN=2;DP=113;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,113:113:99:4048,343,0",2.0,1.0,2.0,113.0,3.0103,0.000,60.0000,27.76,1.270,1/1,"0,113",99.0,"4048,343,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,113.0,1,1,INDEL,113.0,0.000000,1.000000,NaN,NaN,NaN,NaN,1,1,1
53,MTB_anc,169549,.,TGGCGCCGATGCC,T,3250.44,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=0.012;DP=102;Ex...,GT:AD:DP:GQ:PL,"1/1:2,100:102:99:3266,284,0",2.0,1.0,2.0,102.0,0.7918,0.000,60.0000,31.87,1.167,1/1,"2,100",99.0,"3266,284,0",0.012,0.000,1.140,NaN,NaN,NaN,2,2.0,100.0,1,1,INDEL,102.0,0.019608,0.980392,NaN,NaN,NaN,NaN,1,1,1
64,MTB_anc,208316,.,A,AT,13871.13,PASS,AC=2;AF=1.00;AN=2;DP=180;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,180:180:99:1|1:208316_A_AT:8184,551,0:20...",2.0,1.0,2.0,180.0,3.0103,0.000,60.0000,30.56,1.633,1|1,"0,180",99.0,"8184,551,0",NaN,NaN,NaN,1|1,208316_A_AT,208316,2,0.0,180.0,1,1,INDEL,180.0,0.000000,1.000000,NaN,NaN,NaN,NaN,3,3,3
65,MTB_anc,208320,.,GC,G,14074.09,PASS,AC=2;AF=1.00;AN=2;DP=180;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,180:180:99:1|1:208316_A_AT:8301,560,0:20...",2.0,1.0,2.0,180.0,3.0103,0.000,60.0000,33.63,1.876,1|1,"0,180",99.0,"8301,560,0",NaN,NaN,NaN,1|1,208316_A_AT,208316,2,0.0,180.0,1,1,INDEL,180.0,0.000000,1.000000,NaN,NaN,NaN,NaN,3,3,3
75,MTB_anc,234494,.,T,TGC,2825.13,PASS,AC=2;AF=1.00;AN=2;DP=22;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,21:22:78:1142,78,0",2.0,1.0,2.0,22.0,3.0103,0.000,60.0000,30.82,0.760,1/1,"0,21",78.0,"1142,78,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,21.0,1,1,INDEL,21.0,0.000000,1.000000,NaN,NaN,NaN,NaN,1,1,1
84,MTB_anc,293704,.,CT,C,2684.44,PASS,AC=2;AF=1.00;AN=2;DP=83;ExcessHet=0.7918;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,83:83:99:2700,250,0",2.0,1.0,2.0,83.0,0.7918,0.000,60.0000,32.34,1.395,1/1,"0,83",99.0,"2700,250,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,83.0,1,1,INDEL,83.0,0.000000,1.000000,NaN,NaN,NaN,NaN,1,1,1
98,MTB_anc,335810,.,CCCGCCGGCGCCGCCGTTG,C,632.06,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=1.86;DP=8;Exces...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,8:8:24:1|1:335803_G_C:286,24,0:335803",2.0,1.0,2.0,8.0,3.0103,17.374,50.5500,27.48,2.073,1|1,"0,8",24.0,"286,24,0",1.860,-1.548,0.223,1|1,335803_G_C,335803,2,0.0,8.0,1,1,INDEL,8.0,0.000000,1.000000,NaN,NaN,NaN,NaN,2,2,2


In [146]:
indel[indel.POS == 467508]

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,TYPE,dp,aF,AF,snp_left_distance,snp_right_distance,indel_right_distance,indel_left_distance,window_10,window_20,window_30
208,MTB_anc,467508,.,C,CG,6358.13,PASS,AC=2;AF=1.00;AN=2;DP=76;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,76:76:99:1|1:467504_G_A:3599,244,0:467504",2.0,1.0,2.0,76.0,3.0103,0.0,60.0,31.88,0.833,1|1,"0,76",99.0,"3599,244,0",NaN,NaN,NaN,1|1,467504_G_A,467504,2,0.0,76.0,1,1,INDEL,76.0,0.0,1.0,NaN,NaN,NaN,NaN,3,4,5


In [73]:
indel[indel.TYPE == "INDEL"].head(10)

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,TYPE,dp,aF,AF,snp_left_distance,snp_right_distance,window_10,window_20,window_30
25,MTB_anc,71584,.,C,CCGAGCGCTGTTCTGGCGCTAATCTGACGCTAGAATAG,6267.13,PASS,AC=2;AF=1.00;AN=2;DP=53;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,53:53:99:3299,212,0",2.0,1.0,2.0,53.0,3.0103,0.000,60.0000,30.02,3.352,1/1,"0,53",99.0,"3299,212,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,53.0,1,1,INDEL,53.0,0.000000,1.000000,NaN,NaN,1,1,1
27,MTB_anc,79504,.,TCGGTGGACC,T,3698.09,PASS,"AC=0,2;AF=0.00,1.00;AN=2;DP=59;ExcessHet=3.010...",GT:AD:DP:GQ:PL,"2/2:0,0,59:59:99:2621,2621,2621,181,181,0",0.0,2.0,0.0,59.0,2.0000,59.000,3.0103,0.00,60.000,2/2,"0,0,59",99.0,"2621,2621,2621,181,181,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,59.0,2,2,INDEL,59.0,0.000000,1.000000,NaN,NaN,1,1,1
44,MTB_anc,125830,.,G,GA,5951.13,PASS,AC=2;AF=1.00;AN=2;DP=105;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,105:105:99:3399,316,0",2.0,1.0,2.0,105.0,3.0103,0.000,60.0000,32.17,1.142,1/1,"0,105",99.0,"3399,316,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,105.0,1,1,INDEL,105.0,0.000000,1.000000,NaN,NaN,1,1,1
46,MTB_anc,131174,.,T,TG,6359.13,PASS,AC=2;AF=1.00;AN=2;DP=113;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,113:113:99:4048,343,0",2.0,1.0,2.0,113.0,3.0103,0.000,60.0000,27.76,1.270,1/1,"0,113",99.0,"4048,343,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,113.0,1,1,INDEL,113.0,0.000000,1.000000,NaN,NaN,1,1,1
53,MTB_anc,169549,.,TGGCGCCGATGCC,T,3250.44,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=0.012;DP=102;Ex...,GT:AD:DP:GQ:PL,"1/1:2,100:102:99:3266,284,0",2.0,1.0,2.0,102.0,0.7918,0.000,60.0000,31.87,1.167,1/1,"2,100",99.0,"3266,284,0",0.012,0.000,1.140,NaN,NaN,NaN,2,2.0,100.0,1,1,INDEL,102.0,0.019608,0.980392,NaN,NaN,1,1,1
64,MTB_anc,208316,.,A,AT,13871.13,PASS,AC=2;AF=1.00;AN=2;DP=180;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,180:180:99:1|1:208316_A_AT:8184,551,0:20...",2.0,1.0,2.0,180.0,3.0103,0.000,60.0000,30.56,1.633,1|1,"0,180",99.0,"8184,551,0",NaN,NaN,NaN,1|1,208316_A_AT,208316,2,0.0,180.0,1,1,INDEL,180.0,0.000000,1.000000,NaN,NaN,3,3,3
65,MTB_anc,208320,.,GC,G,14074.09,PASS,AC=2;AF=1.00;AN=2;DP=180;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,180:180:99:1|1:208316_A_AT:8301,560,0:20...",2.0,1.0,2.0,180.0,3.0103,0.000,60.0000,33.63,1.876,1|1,"0,180",99.0,"8301,560,0",NaN,NaN,NaN,1|1,208316_A_AT,208316,2,0.0,180.0,1,1,INDEL,180.0,0.000000,1.000000,NaN,NaN,3,3,3
75,MTB_anc,234494,.,T,TGC,2825.13,PASS,AC=2;AF=1.00;AN=2;DP=22;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,21:22:78:1142,78,0",2.0,1.0,2.0,22.0,3.0103,0.000,60.0000,30.82,0.760,1/1,"0,21",78.0,"1142,78,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,21.0,1,1,INDEL,21.0,0.000000,1.000000,NaN,NaN,1,1,1
84,MTB_anc,293704,.,CT,C,2684.44,PASS,AC=2;AF=1.00;AN=2;DP=83;ExcessHet=0.7918;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,83:83:99:2700,250,0",2.0,1.0,2.0,83.0,0.7918,0.000,60.0000,32.34,1.395,1/1,"0,83",99.0,"2700,250,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,83.0,1,1,INDEL,83.0,0.000000,1.000000,NaN,NaN,1,1,1
98,MTB_anc,335810,.,CCCGCCGGCGCCGCCGTTG,C,632.06,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=1.86;DP=8;Exces...,GT:AD:DP:GQ:PGT:PID:PL:PS,"1|1:0,8:8:24:1|1:335803_G_C:286,24,0:335803",2.0,1.0,2.0,8.0,3.0103,17.374,50.5500,27.48,2.073,1|1,"0,8",24.0,"286,24,0",1.860,-1.548,0.223,1|1,335803_G_C,335803,2,0.0,8.0,1,1,INDEL,8.0,0.000000,1.000000,NaN,NaN,2,2,2


In [76]:
indel[indel.window_10 > 4].head(10)

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,TYPE,dp,aF,AF,snp_left_distance,snp_right_distance,window_10,window_20,window_30
150,MTB_anc,336687,.,G,A,2442.83,PASS,AC=2;AF=1.00;AN=2;BaseQRankSum=-1.034e+00;DP=4...,GT:AD:DP:GQ:PL,"1/1:10,33:43:9:1424,9,0",2.0,1.0,2.0,43.0,3.0103,1.467,55.57,29.41,1.085,1/1,"10,33",9.0,"1424,9,0",-1.034,-0.697,2.400,NaN,NaN,NaN,2,10.0,33.0,1,1,SNP,43.0,0.232558,0.767442,67.0,1.0,5,7,10
151,MTB_anc,336688,.,A,C,3061.13,PASS,AC=2;AF=1.00;AN=2;DP=49;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,41:49:99:2042,141,0",2.0,1.0,2.0,49.0,3.0103,0.000,56.19,31.11,1.997,1/1,"0,41",99.0,"2042,141,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,41.0,1,1,SNP,41.0,0.000000,1.000000,1.0,2.0,5,7,10
152,MTB_anc,336690,.,CT,C,2549.60,PASS,AC=2;AF=1.00;AN=2;DP=43;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,35:43:99:1833,123,0",2.0,1.0,2.0,43.0,3.0103,0.000,56.01,34.66,2.035,1/1,"0,35",99.0,"1833,123,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,35.0,1,1,INDEL,35.0,0.000000,1.000000,NaN,NaN,5,7,10
153,MTB_anc,336693,.,G,GC,2596.34,PASS,AC=2;AF=1.00;AN=2;DP=43;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,35:43:99:1878,126,0",2.0,1.0,2.0,43.0,3.0103,0.000,56.01,29.40,2.035,1/1,"0,35",99.0,"1878,126,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,35.0,1,1,INDEL,35.0,0.000000,1.000000,NaN,NaN,5,7,10
154,MTB_anc,336694,.,A,C,2596.30,PASS,AC=2;AF=1.00;AN=2;DP=44;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,36:44:99:1878,126,0",2.0,1.0,2.0,44.0,3.0103,0.000,56.01,36.76,2.165,1/1,"0,36",99.0,"1878,126,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,36.0,1,1,SNP,36.0,0.000000,1.000000,1.0,4.0,5,7,10
397,MTB_anc,1093978,.,T,TG,586.01,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-1.645e+00;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:1,9:10:15:0|1:1093928_G_A:375,0,15:1093928",1.0,0.5,2.0,10.0,3.0103,3.310,33.67,30.52,0.643,0|1,"1,9",15.0,"375,0,15",-1.645,1.040,-1.645,0|1,1093928_G_A,1093928,2,1.0,9.0,0,1,INDEL,10.0,0.100000,0.900000,NaN,NaN,5,7,8
398,MTB_anc,1093981,.,C,G,543.95,MQ40,AC=1;AF=0.500;AN=2;BaseQRankSum=-9.670e-01;DP=...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:1,8:9:18:0|1:1093928_G_A:333,0,18:1093928",1.0,0.5,2.0,9.0,3.0103,3.680,32.90,28.43,0.916,0|1,"1,8",18.0,"333,0,18",-0.967,0.967,-1.593,0|1,1093928_G_A,1093928,2,1.0,8.0,0,1,SNP,9.0,0.111111,0.888889,3.0,2.0,5,7,8
399,MTB_anc,1093983,.,T,G,543.95,MQ40,AC=1;AF=0.500;AN=2;BaseQRankSum=0.967;DP=9;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:1,8:9:18:0|1:1093928_G_A:333,0,18:1093928",1.0,0.5,2.0,9.0,3.0103,3.680,32.90,30.83,0.916,0|1,"1,8",18.0,"333,0,18",0.967,0.967,-1.593,0|1,1093928_G_A,1093928,2,1.0,8.0,0,1,SNP,9.0,0.111111,0.888889,2.0,2.0,5,7,8
400,MTB_anc,1093985,.,GA,G,543.95,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=0.366;DP=7;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:1,6:7:18:0|1:1093928_G_A:333,0,18:1093928",1.0,0.5,2.0,7.0,3.0103,0.000,33.12,27.64,0.477,0|1,"1,6",18.0,"333,0,18",0.366,0.792,-1.465,0|1,1093928_G_A,1093928,2,1.0,6.0,0,1,INDEL,7.0,0.142857,0.857143,NaN,NaN,5,7,8
401,MTB_anc,1093987,.,T,C,459.93,MQ40,AC=1;AF=0.500;AN=2;BaseQRankSum=0.366;DP=7;Exc...,GT:AD:DP:GQ:PGT:PID:PL:PS,"0|1:1,6:7:24:0|1:1093928_G_A:249,0,24:1093928",1.0,0.5,2.0,7.0,3.0103,0.000,33.12,27.23,0.477,0|1,"1,6",24.0,"249,0,24",0.366,0.792,-1.465,0|1,1093928_G_A,1093928,2,1.0,6.0,0,1,SNP,7.0,0.142857,0.857143,2.0,6.0,5,7,8


In [417]:
indel[~indel.ALT.str.contains(r'(A|T|C|G)')]

/home/laura/env36/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,AC,af,AN,DP,ExcessHet,FS,MQ,QD,SOR,GT,AD,GQ,PL,BaseQRankSum,MQRankSum,ReadPosRankSum,PGT,PID,PS,len_AD,REF_AD,ALT_AD,gt0,gt1,dp,aF,AF
909,MTB_anc,2960249,.,C,*,199.52,PASS,"AC=0,2;AF=0.00,1.00;AN=2;DP=17;ExcessHet=3.010...",GT:AD:DP:GQ:PGT:PID:PL:PS,"2|2:0,0,17:17:51:1|1:2960237_CGCG_C:719,719,71...",0.0,2.0,0.0,17.0,2.0,17.0,3.0103,0.0,60.00,2|2,"0,0,17",51.0,"719,719,719,51,51,0",NaN,NaN,NaN,1|1,2960237_CGCG_C,2960237,2,0.0,17.0,2,2,17.0,0.0,1.0
967,MTB_anc,3119735,.,A,*,1117.55,PASS,"AC=0,2;AF=0.00,1.00;AN=2;BaseQRankSum=2.30;DP=...",GT:AD:DP:GQ:PL,"2/2:0,0,22:22:70:998,1012,1105,70,81,0",0.0,2.0,0.0,22.0,2.3,22.0,3.0103,60.0,19.61,2/2,"0,0,22",70.0,"998,1012,1105,70,81,0",1.0,3.076,0.0,NaN,NaN,NaN,2,0.0,22.0,2,2,22.0,0.0,1.0
968,MTB_anc,3119741,.,T,*,1619.57,PASS,"AC=0,2;AF=0.00,1.00;AN=2;DP=22;ExcessHet=3.010...",GT:AD:DP:GQ:PL,"2/2:0,0,22:22:70:998,1012,1105,70,81,0",0.0,2.0,0.0,22.0,2.0,22.0,3.0103,0.0,60.00,2/2,"0,0,22",70.0,"998,1012,1105,70,81,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,22.0,2,2,22.0,0.0,1.0
1485,MTB_anc,3119738,.,GGT,*,1661.57,PASS,"AC=0,2;AF=0.00,1.00;AN=2;DP=22;ExcessHet=3.010...",GT:AD:DP:GQ:PL,"2/2:0,0,22:22:70:998,1012,1105,70,81,0",0.0,2.0,0.0,22.0,2.0,22.0,3.0103,0.0,60.00,2/2,"0,0,22",70.0,"998,1012,1105,70,81,0",NaN,NaN,NaN,NaN,NaN,NaN,2,0.0,22.0,2,2,22.0,0.0,1.0
